In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## 获取数据集

这里是要把问题所需的训练数据整理出来，基本上就是一个机器学习的数据集获取的过程。

这是土壤湿度W，每月的数据。注意：代码里面用符号H表示，实际上是W。

In [64]:
df = pd.read_csv('./附件3-湿度-预处理.csv', index_col='time')
df_H=df.drop(columns=['month','year'])
df_H.columns

Index(['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)', '200cm湿度(kg/m2)'], dtype='object')

这是土壤蒸发量E，每月的数据，和湿度W是对齐的（数据的时间范围完全相同），注意：代码里面用符号SoilEvap表示，实际上是E。

In [65]:
df = pd.read_csv('./附件4-土壤蒸发量-预处理.csv', index_col='time')
df_SoilEvap=df.drop(columns=['month','year'])
df_SoilEvap.columns

Index(['土壤蒸发量(W/m2)', '土壤蒸发量(mm)'], dtype='object')

这是气候条件，有很多变量，但大部分变量和要预测的湿度相关性不大（见相关性分析）。
筛选了4个相关性比较大的变量：
```
['土壤蒸发量(mm)', '平均露点温度(℃)', '最低气温极值(℃)', '平均最低气温(℃)']
```

In [66]:
df = pd.read_csv('附件8-气候-预处理.csv',index_col='time')
df_Climate=df.drop(columns=['month','year'])

In [67]:
df_All = df_Climate.join([df_H, df_SoilEvap])
# 对所有自变量、因变量合并为一个df

In [68]:
df_All.columns

Index(['平均气温(℃)', '平均最高气温(℃)', '平均最低气温(℃)', '最高气温极值(℃)', '最低气温极值(℃)',
       '平均气温≥18℃的天数', '平均气温≤0℃的天数', '平均露点温度(℃)', '降水量(mm)', '最大单日降水量(mm)',
       '降水天数', '平均海平面气压(hPa)', '最低海平面气压(hPa)', '平均站点气压(hPa)', '平均能见度(km)',
       '最小能见度(km)', '最大能见度(km)', '平均风速(knots)', '平均最大持续风速(knots)',
       '单日最大平均风速(knots)', '10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)',
       '200cm湿度(kg/m2)', '土壤蒸发量(W/m2)', '土壤蒸发量(mm)'],
      dtype='object')

In [69]:
df_All.index

Index(['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01',
       '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01',
       ...
       '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01',
       '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01'],
      dtype='object', name='time', length=127)

In [70]:
Y_COLS=['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)','200cm湿度(kg/m2)']
X_COLS=['土壤蒸发量(mm)', '降水量(mm)', '平均气温(℃)']

这里定义了4个因变量和3个自变量。

In [71]:
df=df_All[Y_COLS+X_COLS].dropna()
df.columns,df.index

(Index(['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)', '200cm湿度(kg/m2)',
        '土壤蒸发量(mm)', '降水量(mm)', '平均气温(℃)'],
       dtype='object'),
 Index(['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01',
        '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01',
        ...
        '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
        '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01'],
       dtype='object', name='time', length=123))

In [72]:
df.to_csv('问题2-原训练数据.csv')

这里获取了模型预测所需要的数据，即3个自变量和3个因变量。

In [73]:
scaler=MinMaxScaler()
# df_val=scaler.fit_transform(df)
scaler.fit(df)
df_val=scaler.transform(df)
df=pd.DataFrame(data=df_val, index=df.index, columns=df.columns)
df.to_csv('问题2-训练数据-归一化.csv')
import pickle
pickle.dump(scaler, P('问题2-MinMaxScaler-训练数据.pkl').open('wb'))
df

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),200cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
time,,,,,,,
2012-01-01,0.328250,0.036261,0.008417,1.000000,0.008720,0.002568,0.008070
2012-02-01,0.269663,0.036261,0.008417,1.000000,0.015132,0.002568,0.113180
2012-03-01,0.237560,0.036261,0.008417,1.000000,0.042062,0.024490,0.335609
2012-04-01,0.186998,0.031885,0.007829,1.000000,0.177738,0.006840,0.626319
2012-05-01,0.364366,0.008753,0.004306,1.000000,0.457297,0.056964,0.785020
...,...,...,...,...,...,...,...
2021-11-01,0.342697,0.702094,0.999609,0.002451,0.030008,0.595275,0.360852
2021-12-01,0.280899,0.701157,1.000000,0.000000,0.016927,0.217316,0.225947
2022-01-01,0.225522,0.701157,1.000000,0.000000,0.004360,0.681001,0.129112


In [74]:
scaler.scale_, scaler.data_range_

(array([0.08025682, 0.03125977, 0.0195733 , 0.24509804, 0.0256476 ,
        0.00112134, 0.02069108]),
 array([ 12.46,  31.99,  51.09,   4.08,  38.99, 891.79,  48.33]))

对变量进行最小最大值归一化（MinMaxScale），把变量的范围映射到0-1区间。

In [75]:
df.index

Index(['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01',
       '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01',
       ...
       '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
       '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01'],
      dtype='object', name='time', length=123)

In [76]:
px.line(df).update_layout(xaxis_title='时间',yaxis_title='取值',legend_title='变量', title='问题2 自变量和因变量的时序图')